# Registro de detecciones

Este cuaderno ensaya el registro de detecciones en videos con el objetivo es poder hacer consultas en las bases de datos para:

- Identificar momentos de interés (presencia de objetos buscados)
- Análisis de detalle usando la información de las detecciones.

Salidas:
- Archivos CSV.
- Series temporales.
- InfluxDB.

In [56]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../videoanalytics/src")
import videoanalytics
videoanalytics.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'0.0.2'

In [57]:
from datetime import datetime

## Chequeo de disponibilidad de HW / GPU

In [58]:
!nvidia-smi

Thu Aug 12 13:37:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Quadro M...  On   | 00000000:02:00.0 Off |                  N/A |
| 46%   30C    P8    11W / 120W |     62MiB /  8126MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA Quadro R...  On   | 00000000:81:00.0 Off |                  Off |
| 34%   

Detectadas por tensorflow en conda (FIXME).

In [59]:
import tensorflow as tf
print("GPUs disponibles: ", len(tf.config.list_physical_devices('GPU')))

GPUs disponibles:  0


## Datos de entrada y modelos

In [60]:
WORKSPACE_PATH="../"
WORKSPACE_DATA_PATH=WORKSPACE_PATH+"/data/"
INPUT_VIDEO_PATH=WORKSPACE_DATA_PATH+"media/videos-youtube/"
!ls {INPUT_VIDEO_PATH}

'buque EL MARISCO II ,proceso de captura merluz.mp4'
'LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4'
'Maniobra de pesca de merluza en mares argentino.mp4'
 maniobr.mp4
'Pesca artesanal de anchoita en mar del plat.mp4'
'pesca de altura en el mar argentino,pesquero.mp4'
'pesca de langostino,buque Alve.mp4'
'Pesca de langostino en aguas nacionale.mp4'
'pesca de langostino en rawson chubut con barco combenciona.mp4'
'PESCA DE LANGOSTINO RW CHUBU.mp4'
'PESCA EN RAWSO.mp4'
'Un dia de pesca merluza  Buque Paola.mp4'
'WhatsApp Video 2021-07-25 at 10.21.14 PM.mp4'
'WhatsApp Video 2021-07-25 at 9.24.42 PM.mp4'


Repositorio de modelos (local).

In [61]:
MODELS_PATH=WORKSPACE_DATA_PATH+"models/"
!ls {MODELS_PATH}

feature_extractor_ll.tar.gz  kaggle-fisheries-yolo  mirtar-yolo  README.md


In [62]:
!tree {MODELS_PATH}/kaggle-fisheries-yolo

..//data/models//kaggle-fisheries-yolo
├── checkpoints
│   └── yolo-416
│       ├── assets
│       ├── saved_model.pb
│       └── variables
│           ├── variables.data-00000-of-00001
│           └── variables.index
├── kaggle-fisheries-perf-report.txt
├── kaggle-fisheries-yolo4.cfg
├── kaggle-fisheries-yolo4.weights
├── obj.names
└── README.md

4 directories, 8 files


In [63]:
!tree {MODELS_PATH}/mirtar-yolo

..//data/models//mirtar-yolo
├── checkpoints
│   └── yolo-416
│       ├── assets
│       ├── saved_model.pb
│       └── variables
│           ├── variables.data-00000-of-00001
│           └── variables.index
├── mirtar-yolo4.cfg
├── mirtar-yolo4.weights
├── obj.names
└── README.md

4 directories, 7 files


## Definición de cadenas de procesamiento

### Detección con una única instancia del detector



In [64]:
from videoanalytics.pipeline import Pipeline
from videoanalytics.pipeline.sources import VideoReader
from videoanalytics.pipeline.sinks import VideoWriter

from videoanalytics.pipeline.sinks.object_detection import DetectionsAnnotator, DetectionsCSVWriter
from videoanalytics.pipeline.sinks.object_detection.yolo4 import YOLOv4DetectorTF

def make_detection_pipeline(input_video_filename,
                            output_csv_filename,
                            output_video_filename,
                            detector_model_weights_filename,
                            detector_model_classes_filename,
                            start_frame=0,
                            max_frames=None):
    context = {}
    pipeline = Pipeline()

    pipeline.add_component( VideoReader( "input",context,
                     video_path=input_video_filename,
                     start_frame=start_frame,
                     max_frames=max_frames))    
    
    pipeline.add_component( YOLOv4DetectorTF("detector",
                                             context,
                                             weights_filename=detector_model_weights_filename) )
        
    pipeline.add_component( DetectionsCSVWriter("det_csv_writer",context,filename=output_csv_filename) )
    
    pipeline.add_component( DetectionsAnnotator("annotator",
                                                context,
                                                class_names_filename=detector_model_classes_filename,
                                                show_label=True) )
    
    pipeline.add_component(VideoWriter("writer",context,filename=output_video_filename))
    
    pipeline.set_connections([
        ("input", "detector"),
        ("detector", "det_csv_writer"),
        ("detector", "annotator"),
        ("annotator", "writer")
    ])    
    
    return context,pipeline

## Batch job

Directorio de resultados.

In [65]:
!ls {WORKSPACE_DATA_PATH}/output

kaggle-fisheries-yolo  mirtar-yolo


In [66]:
INPUT_VIDEOS = [
    (
        'buque EL MARISCO II ,proceso de captura merluz.mp4',
        None,
    ),
    
    (
        'LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4',
        None,        
    ),
    
    (
        'Maniobra de pesca de merluza en mares argentino.mp4',
        None,
    ),
    
    ( 
        'maniobr.mp4',
        None
    ),
    
    (
        'Pesca artesanal de anchoita en mar del plat.mp4',
        None
    ),
    
    (
        'pesca de altura en el mar argentino,pesquero.mp4',
        None
    ),
    
    (
        'pesca de langostino,buque Alve.mp4',
        None
    ),
    
    (
        'Pesca de langostino en aguas nacionale.mp4',
        None
    ),
    
    (
        'pesca de langostino en rawson chubut con barco combenciona.mp4',
        None,
    ),
    
    (
        'PESCA DE LANGOSTINO RW CHUBU.mp4',
        None,
    ),
    
    (
        'PESCA EN RAWSO.mp4',
        None
    ),
    
    (
        'Un dia de pesca merluza  Buque Paola.mp4',
        None
    ),
    
    (
        'WhatsApp Video 2021-07-25 at 10.21.14 PM.mp4',
        None
    ),
    
    (
        'WhatsApp Video 2021-07-25 at 9.24.42 PM.mp4',
        None
    )
]

In [67]:
from os import path

def object_detection_batch_job(input_videos,obj_det_model, output_path,dry_run=False):
    
    t0 = datetime.now()
    print("Inicio de proceso: ", t0)
    
    for i,v in enumerate(input_videos):
        input_video_filename=v[0]        
        output_video_filename=output_path+"job{:d}.mp4".format(i)
        output_csv_filename=output_path+"job{:d}.csv".format(i)

        output_csv_filename

        print("Job {}. Procesando {}.".format(i, input_video_filename))
        
        if path.exists(output_video_filename):
            print("El archivo a generar {} ya existe. Pasando al siguiente".format(output_video_filename))    
            continue

        # FIXME, por ahora son fijos
        params=v[1]

        detector_model_weights_filename=MODELS_PATH+"/{}/checkpoints/yolo-416/".format(obj_det_model)
        detector_model_classes_filename=MODELS_PATH+"/{}/obj.names".format(obj_det_model)

        

        if not dry_run:
            context,pipeline=make_detection_pipeline(
                input_video_filename=INPUT_VIDEO_PATH+input_video_filename,
                output_csv_filename=output_csv_filename,
                output_video_filename=output_video_filename,
                detector_model_weights_filename=detector_model_weights_filename,
                detector_model_classes_filename=detector_model_classes_filename,
                start_frame=0,
                max_frames=None
            )

            pipeline.execute()
            print("Tiempo total de ejecución [s]:", pipeline.get_total_execution_time())
            print("Video de salida guardado en {}".format(output_video_filename))
    
    t1 = datetime.now()
    print("Fin del proceso:",t1)        
    print("Tiempo transcurrido:",t1-t0)

### Job con dataset de kaggle-fisheries

In [99]:
input_videos = INPUT_VIDEOS
output_base_path=WORKSPACE_DATA_PATH+"/output/"
obj_det_model = "kaggle-fisheries-yolo/"
output_path = output_base_path+obj_det_model

In [100]:
#!rm -rf {output_path} 
#!mkdir {output_path}

In [ ]:
object_detection_batch_job(input_videos,obj_det_model, output_path)

Inicio de proceso:  2021-08-13 03:44:49.026231
Job 0. Procesando buque EL MARISCO II ,proceso de captura merluz.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 2250.782152853033
Video de salida guardado en ..//data//output/kaggle-fisheries-yolo/job0.mp4
Job 1. Procesando LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 6788.504858753004
Video de salida guardado en ..//data//output/kaggle-fisheries-yolo/job1.mp4
Job 2. Procesando Maniobra de pesca de merluza en mares argentino.mp4.


...

  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 5677.020367505029
Video de salida guardado en ..//data//output/kaggle-fisheries-yolo/job3.mp4
Job 4. Procesando Pesca artesanal de anchoita en mar del plat.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

Tiempo total de ejecución [s]: 5059.284656103991
Video de salida guardado en ..//data//output/kaggle-fisheries-yolo/job4.mp4
Job 5. Procesando pesca de altura en el mar argentino,pesquero.mp4.


  0%|          | 0/100.0 [00:00<?, ?it/s]

### Job con dataset Mirta-R

In [70]:
input_videos = INPUT_VIDEOS
output_base_path=WORKSPACE_DATA_PATH+"/output/"
obj_det_model = "mirtar-yolo"
output_path = output_base_path+obj_det_model+"/"

In [71]:
# Reset
#!rm -rf {output_path}
#!mkdir {output_path}

In [98]:
object_detection_batch_job(input_videos,obj_det_model, output_path)

Inicio de proceso:  2021-08-13 03:44:48.953189
Job 0. Procesando buque EL MARISCO II ,proceso de captura merluz.mp4.
El archivo a generar ..//data//output/mirtar-yolo/job0.mp4 ya existe. Pasando al siguiente
Job 1. Procesando LANGOSTINOS  PUERTO RAWSON  ENERO 201.mp4.
El archivo a generar ..//data//output/mirtar-yolo/job1.mp4 ya existe. Pasando al siguiente
Job 2. Procesando Maniobra de pesca de merluza en mares argentino.mp4.
El archivo a generar ..//data//output/mirtar-yolo/job2.mp4 ya existe. Pasando al siguiente
Job 3. Procesando maniobr.mp4.
El archivo a generar ..//data//output/mirtar-yolo/job3.mp4 ya existe. Pasando al siguiente
Job 4. Procesando Pesca artesanal de anchoita en mar del plat.mp4.
El archivo a generar ..//data//output/mirtar-yolo/job4.mp4 ya existe. Pasando al siguiente
Job 5. Procesando pesca de altura en el mar argentino,pesquero.mp4.
El archivo a generar ..//data//output/mirtar-yolo/job5.mp4 ya existe. Pasando al siguiente
Job 6. Procesando pesca de langostino,b

#### Análisis de resultados

In [75]:
import pandas as pd

job_id=3
df = pd.read_csv(output_path+"/job{}.csv".format(job_id),
                 names=["class_idx","x","y","w","h","score","drop"],index_col=0).iloc[:, :-1]
df.head()

,class_idx,x,y,w,h,score
0,0,286.0,119.0,51.0,53.0,0.998765
0,0,221.0,123.0,52.0,47.0,0.998040
1,0,286.0,119.0,51.0,53.0,0.998765
1,0,221.0,123.0,52.0,47.0,0.998048
2,0,286.0,119.0,50.0,53.0,0.998751


In [96]:
df.class_idx.value_counts()

0    78821
2    33213
1     3164
Name: class_idx, dtype: int64